In [1]:
# import pandas and gurobi
import pandas as pd
from gurobipy import *
import numpy as np


In [2]:
# read in csv
r = pd.read_csv("/Users/zachwayne/Documents/GitHub/15093-project/2019_monthly_data/monthly_returns_2019.csv")
s = pd.read_csv("/Users/zachwayne/Documents/GitHub/15093-project/2019_monthly_data/sectors_2019.csv")
p = pd.read_csv("/Users/zachwayne/Documents/GitHub/15093-project/2019_monthly_data/monthly_prices_2019.csv")
cor = pd.read_csv("/Users/zachwayne/Documents/GitHub/15093-project/2019_monthly_data/corr_2019.csv")
cov = pd.read_csv("/Users/zachwayne/Documents/GitHub/15093-project/2019_monthly_data/cov_2019.csv")
cov = cov.drop(columns=['Unnamed: 0'])
s = s.drop(columns=['Symbol'])
p = p.drop(columns=['Date'])
r = r.drop(columns=['Date'])
cor = cor.drop(columns=['Unnamed: 0'])

In [3]:
# make model
stocks = len(s)
k = 20

In [4]:
m = Model('portfolio')

# Add matrix variable for the stocks
x = m.addMVar(stocks)
z = m.addVars(stocks, vtype=GRB.BINARY, name="z")
b = m.addVars(stocks, vtype=GRB.INTEGER, lb=0, name="x")

# Objective is to minimize risk (squared).  This is modeled using the
# covariance matrix, which measures the historical correlation between stocks
portfolio_risk = x @ np.matrix(cov) @ x
ret = np.matrix(r.iloc[0]) @ x
m.setObjective(portfolio_risk - ret, GRB.MINIMIZE)

# Fix budget with a constraint
m.addConstr(x.sum() == 1, 'budget')
# return is at least 0.1
m.addConstr(portfolio_risk <= 0.001, 'return')
m.addConstrs((x[i] <= z[i] for i in range(stocks)), "w<=z")
m.addConstrs((100*x[i] >= z[i] for i in range(stocks)), "w>=z")
m.addConstr(quicksum(z[i] for i in range(stocks)) == k, "k")
m.addConstr(quicksum(b[i]*p.iloc[0,i] for i in range(stocks)) <= 300000, "budget")
m.addConstrs((b[i] >= z[i] for i in range(stocks)), "x>=fvfav")
m.addConstrs((x[i] == (b[i]*p.iloc[0,i])/300000 for i in range(stocks)), "w")
m.addConstrs((quicksum(z[i]*s.iloc[i,j] for i in range(stocks)) <= 3 for j in range(len(s.columns))), "sector")
#m.Params.MIPGap = 0.05

m.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1922 rows, 1431 columns and 5724 nonzeros
Model fingerprint: 0xdc7c42a4
Model has 114003 quadratic objective terms
Model has 1 quadratic constraint
Variable types: 477 continuous, 954 integer (477 binary)
Coefficient statistics:
  Matrix range     [2e-05, 3e+03]
  QMatrix range    [1e-08, 1e-01]
  Objective range  [6e-04, 3e-01]
  QObjective range [2e-08, 2e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+05]
  QRHS range       [1e-03, 1e-03]
Presolve time: 0.08s
Presolved: 1922 rows, 1431 columns, 5724 nonzeros
Presolved model has 114003 quadratic objective terms
Presolved model has 1 quadratic constraint(s)
Variable types: 477 continuous, 954 integer (477 binary)

Root relaxation: objective -6.185504e-02, 2215 iter

In [5]:
# print optimal objective
print("Optimal objective: " + str(m.objVal))
  

Optimal objective: -0.0454967256506353


In [6]:
# print optimal x
total = 0
for v in m.getVars():
    if v.x > 0 and v.varName[0] == 'x':
        print(p.columns[int(v.varName.replace("x[","").replace("]",""))] + ": " + str(v.x))
        total += v.x*p.iloc[0,int(v.varName.replace("x[","").replace("]",""))]

print(total)

ABBV: 374.0
AZO: 4.0
BMY: 69.0
AVGO: 332.0
CHD: 49.0
CME: 19.0
DLTR: 252.0
EIX: 62.0
RE: 15.0
HIG: 72.0
HSY: 31.0
IFF: 24.0
KMB: 31.0
MTCH: 1761.0
NEM: 532.0
NRG: 82.0
ORLY: 9.0
O: 53.0
RMD: 212.0
TTWO: 29.0
299999.7555961609
